In [3]:
using Pkg
Pkg.activate("..")
using ElementalFunctionsTests

  Activating project at `~/Documents/Academia/taylor/ElementalFunctionsTests`


In [4]:
using ProgressMeter
using JLD2

In [3]:
function buckets(r) 
    return zeros(r.len-1)
end

buckets (generic function with 1 method)

In [4]:
function add2bucket!(h, r, x)
  if x<r[1] || abs(x) == 0;
    return
  end
  for i in 2:r.len
    if x<r[i] 
      h[i-1]+=1
      return
    end
  end
end

add2bucket! (generic function with 1 method)

In [5]:
const r = range(-2, 2, length=41)
errors = []

Any[]

In [16]:
for f in mpfrfunctions
    x = nextfloat(-Inf32)
    maxerror = ElementalFunctionsTests.Error(0.0, 0.0)
    h = buckets(r)
    
    f_c = eval(:(@cfunction($f, Cfloat, (Cfloat,))))
    f_mpfr = mpfrfun(string(f))

    dom = domain(Symbol(f))
    
    prog = Progress(4278190078, showspeed=true, dt=1.0, desc=string(f))

    while(!isinf(x))
        next!(prog)
        
        if x < dom[1]
            x = nextfloat(x)
            continue
        elseif x > dom[2]
            break;
        end

        err = ulperror32(f_c, f_mpfr, x)
        add2bucket!(h, r, err)
        if err>maxerror.err
          maxerror = ElementalFunctionsTests.Error(x, err)
        end
        x = nextfloat(x)
    end
    finish!(prog)
    push!(errors, TestsResults(Symbol(f),r,h, maxerror, 0))
    @save "float32results.jld2" errors
end

cos 100%|██████████████████████████████████| Time: 4:19:18 ( 3.64 μs/it)
sin 100%|██████████████████████████████████| Time: 4:35:16 ( 3.86 μs/it)
tan 100%|██████████████████████████████████| Time: 4:53:32 ( 4.12 μs/it)
cospi 100%|████████████████████████████████| Time: 3:37:54 ( 3.06 μs/it)
sinpi 100%|████████████████████████████████| Time: 6:58:04 ( 5.86 μs/it)m
acos 100%|█████████████████████████████████| Time: 3:04:00 ( 2.58 μs/it)
asin 100%|█████████████████████████████████| Time: 2:37:08 ( 2.20 μs/it)
atan 100%|█████████████████████████████████| Time: 4:27:16 ( 3.75 μs/it)
csc 100%|██████████████████████████████████| Time: 4:54:15 ( 4.13 μs/it)
sec 100%|██████████████████████████████████| Time: 4:30:06 ( 3.79 μs/it)
cot 100%|██████████████████████████████████| Time: 5:27:52 ( 4.60 μs/it)
cosh 100%|█████████████████████████████████| Time: 3:48:36 ( 3.21 μs/it)
sinh 100%|█████████████████████████████████| Time: 3:48:56 ( 3.21 μs/it)
tanh 100%|█████████████████████████████████| Time:

In [27]:
map(x->(x.f, x.maxError), errors)

25-element Vector{Tuple{Symbol, ElementalFunctionsTests.Error}}:
 (:cos, ElementalFunctionsTests.Error(-1.2022152174610467e32, 0.5008998416089551))
 (:sin, ElementalFunctionsTests.Error(-2.5760141952836207e31, 0.5009001286718487))
 (:tan, ElementalFunctionsTests.Error(-6.4672178176e10, 0.7998569047278248))
 (:cospi, ElementalFunctionsTests.Error(-0.23761744797229767, 0.5008956949810429))
 (:sinpi, ElementalFunctionsTests.Error(-0.43150874972343445, 0.5008975906249855))
 (:acos, ElementalFunctionsTests.Error(-0.5320551991462708, 0.8987813930645389))
 (:asin, ElementalFunctionsTests.Error(-0.5007328391075134, 0.7298913713207066))
 (:atan, ElementalFunctionsTests.Error(0.692195475101471, 0.8521076359054892))
 (:csc, ElementalFunctionsTests.Error(1.7658314e7, 1.4980947537083138))
 (:sec, ElementalFunctionsTests.Error(-1.0322423130356704e34, 1.4972213015214857))
 (:cot, ElementalFunctionsTests.Error(-4.2137537847292745e34, 1.7966313621503505))
 (:cosh, ElementalFunctionsTests.Error(-89.4159

In [5]:
@load "float32results.jld2"

1-element Vector{Symbol}:
 :errors

In [19]:
Float32(errors[12].maxError.x)

-89.41599f0

In [20]:
widen(ans)

-89.4159927368164